In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.

X = df.drop(columns=["koi_disposition"])
y = df["koi_disposition"]

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
best_features = SelectKBest(f_classif, k=20)
best_features.fit(X, y)
selected_features = best_features.fit_transform(X, y)
best_features.get_support()

array([ True,  True,  True,  True, False, False, False, False,  True,
        True, False, False, False,  True,  True,  True,  True, False,
       False, False, False, False,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
       False,  True, False, False])

In [6]:
y = df["koi_disposition"]
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
encoded_y
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

Using TensorFlow backend.


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, one_hot_y, random_state=1)

In [8]:
X_train

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         6.6000000e-02, -1.2300000e-01,  2.9809543e+02],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         7.8000000e-02, -5.2000000e-02,  2.9573535e+02],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         5.4000000e-02, -2.7000000e-02,  2.9218417e+02],
       ...,
       [ 0.0000000e+00,  0.0000000e+00,  1.0000000e+00, ...,
         2.4000000e-02, -5.6000000e-02,  2.9917062e+02],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         1.2600000e-01, -1.0300000e-01,  2.8816470e+02],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         5.5000000e-02, -1.6500000e-01,  2.8547504e+02]])

In [9]:
y_train

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScater model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_train)

# Transform the training and testing data using the X_scaler and y_scaler models
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=60, activation='relu', input_dim=20))
model.add(Dense(units=3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                1260      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 183       
Total params: 1,443
Trainable params: 1,443
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 5243 samples
Epoch 1/100
5243/5243 - 1s - loss: 0.7752 - accuracy: 0.6363
Epoch 2/100
5243/5243 - 0s - loss: 0.4778 - accuracy: 0.7877
Epoch 3/100
5243/5243 - 0s - loss: 0.4094 - accuracy: 0.7992
Epoch 4/100
5243/5243 - 0s - loss: 0.3899 - accuracy: 0.8043
Epoch 5/100
5243/5243 - 0s - loss: 0.3798 - accuracy: 0.8129
Epoch 6/100
5243/5243 - 0s - loss: 0.3740 - accuracy: 0.8102
Epoch 7/100
5243/5243 - 0s - loss: 0.3668 - accuracy: 0.8175
Epoch 8/100
5243/5243 - 0s - loss: 0.3607 - accuracy: 0.8238
Epoch 9/100
5243/5243 - 0s - loss: 0.3576 - accuracy: 0.8253
Epoch 10/100
5243/5243 - 0s - loss: 0.3529 - accuracy: 0.8268
Epoch 11/100
5243/5243 - 0s - loss: 0.3481 - accuracy: 0.8331
Epoch 12/100
5243/5243 - 0s - loss: 0.3474 - accuracy: 0.8276
Epoch 13/100
5243/5243 - 0s - loss: 0.3435 - accuracy: 0.8365
Epoch 14/100
5243/5243 - 0s - loss: 0.3390 - accuracy: 0.8428
Epoch 15/100
5243/5243 - 0s - loss: 0.3352 - accuracy: 0.8463
Epoch 16/100
5243/5243 - 0s - loss: 0.3344 - accuracy: 0.

In [14]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1 - 0s - loss: 0.2580 - accuracy: 0.8942
Normal Neural Network - Loss: 0.25569766596198357, Accuracy: 0.8941647410392761


# Adding a Hidden Layer Tuning

Make a Deep Learning Model

In [16]:
deep_model = Sequential()
deep_model.add(Dense(units=60, activation='relu', input_dim=20))
deep_model.add(Dense(units=60, activation='relu'))
deep_model.add(Dense(units=3, activation='softmax'))
deep_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 60)                1260      
_________________________________________________________________
dense_6 (Dense)              (None, 60)                3660      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 183       
Total params: 5,103
Trainable params: 5,103
Non-trainable params: 0
_________________________________________________________________


In [18]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 5243 samples
Epoch 1/100
5243/5243 - 1s - loss: 0.5773 - accuracy: 0.7011
Epoch 2/100
5243/5243 - 0s - loss: 0.3948 - accuracy: 0.7910
Epoch 3/100
5243/5243 - 0s - loss: 0.3785 - accuracy: 0.8032
Epoch 4/100
5243/5243 - 0s - loss: 0.3685 - accuracy: 0.8121
Epoch 5/100
5243/5243 - 0s - loss: 0.3610 - accuracy: 0.8207
Epoch 6/100
5243/5243 - 0s - loss: 0.3520 - accuracy: 0.8304
Epoch 7/100
5243/5243 - 0s - loss: 0.3477 - accuracy: 0.8327
Epoch 8/100
5243/5243 - 0s - loss: 0.3417 - accuracy: 0.8394
Epoch 9/100
5243/5243 - 0s - loss: 0.3387 - accuracy: 0.8360
Epoch 10/100
5243/5243 - 0s - loss: 0.3296 - accuracy: 0.8442
Epoch 11/100
5243/5243 - 0s - loss: 0.3256 - accuracy: 0.8491
Epoch 12/100
5243/5243 - 0s - loss: 0.3238 - accuracy: 0.8554
Epoch 13/100
5243/5243 - 0s - loss: 0.3211 - accuracy: 0.8539
Epoch 14/100
5243/5243 - 0s - loss: 0.3209 - accuracy: 0.8531
Epoch 15/100
5243/5243 - 0s - loss: 0.3130 - accuracy: 0.8610
Epoch 16/100
5243/5243 - 0s - loss: 0.3101 - accuracy: 0.

In [20]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1 - 0s - loss: 0.2467 - accuracy: 0.9033
Deep Neural Network - Loss: 0.24533817625564078, Accuracy: 0.9033181071281433


# Save the Model

In [24]:
model.save("deep_model.h5")